In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
from utils import (chart_price, plot, v_backtester, c_backtester, perf, perf_var, 
                   breakout_strategy, bootstrap, m_proc, sampler)
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
import matplotlib.pyplot as plt
from typing import NamedTuple
from collections import namedtuple
%matplotlib inline

In [2]:
from datastore_pytables import Store

In [3]:
store = Store()
store.keys()

['/cont/min/CL_20191120_NYMEX_USD',
 '/cont/min/ES_20191220_GLOBEX_USD',
 '/cont/min/GC_20191227_NYMEX_USD',
 '/cont/min/GE_20191216_GLOBEX_USD',
 '/cont/min/NKD_20191212_GLOBEX_USD',
 '/cont/min/NQ_20191220_GLOBEX_USD',
 '/cont/min/YM_20191220_ECBOT_USD',
 '/cont/min/ZB_20191219_ECBOT_USD',
 '/cont/min/ZF_20191231_ECBOT_USD',
 '/cont/min/ZN_20191219_ECBOT_USD']

In [4]:
contract = store.read('/cont/min/NQ_20191220_GLOBEX_USD').sort_index()

In [5]:
table = sampler(contract, start = '20180102', period_length=50, paths=100)

In [6]:
func = partial(breakout_strategy, 
                  time_int = 23,
                  periods = [5, 10, 20, 40, ],#80, 160,],
                  ema_fast = 10,
                  ema_slow = 120,
                  atr_periods = 80,
                  sl_atr = 1,
                  )

In [7]:
#results = [func(i) for i in table]

In [8]:
results = m_proc(table, func)

In [9]:
res = pd.DataFrame([perf(i, output=False).stats for i in results])

In [10]:
res[res['Annual return'] > 0]['Annual return'].count()

88

In [11]:
res.sort_values('Annual return')

,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,...,Avg gain/loss ratio,Position EV,Positions per day,Days per position,Actual avg. duration,Days,Positions,Trades,Monthly EV,Annual EV
80,-0.351059,-0.079064,0.157501,-2.666122,-3.777714,0.432258,-0.092929,0.637346,-3.595987,0.521465,...,1.242064,-6.048387,1.937500,0.516129,05:48:00,48,93,186,-246.093750,-2953.125000
67,-0.247015,-0.051539,0.117667,-2.351775,-3.522785,0.838564,-0.070119,0.637452,-2.848300,-1.177207,...,1.476592,-4.369681,2.000000,0.500000,04:50:00,47,94,188,-183.526596,-2202.319149
14,-0.192984,-0.040018,0.127267,-1.621500,-2.608309,0.790376,-0.073988,0.742409,-2.116623,-0.461472,...,1.625705,-3.260309,2.020833,0.494845,04:09:00,48,97,194,-138.359375,-1660.312500
1,-0.180148,-0.038645,0.124219,-1.537482,-2.798588,0.602698,-0.064371,0.766379,-2.057208,-0.021205,...,1.565719,-3.222826,1.840000,0.543478,04:09:00,50,92,184,-124.530000,-1494.360000
81,-0.166176,-0.034024,0.099328,-1.780393,-2.306429,0.548688,-0.072049,0.740368,-2.474910,0.484708,...,1.306399,-2.755051,2.062500,0.484848,02:56:00,48,99,197,-119.328125,-1431.937500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,0.628983,0.095277,0.120689,4.105793,28.237844,0.870429,-0.022274,2.185712,8.253417,0.494964,...,1.957022,7.623563,1.851064,0.540230,08:17:00,47,87,174,296.345745,3556.148936
23,0.631441,0.101988,0.140635,3.552594,22.677453,0.891165,-0.027844,1.950992,6.072162,-0.113845,...,2.005864,8.846154,1.560000,0.641026,08:38:00,50,78,155,289.800000,3477.600000
76,0.663615,0.104034,0.155925,3.344106,17.649676,0.828535,-0.037599,1.794536,5.402635,-0.490563,...,2.103762,9.344156,1.571429,0.636364,07:19:00,49,77,154,308.357143,3700.285714
82,0.741483,0.106566,0.136319,4.139917,35.177303,0.858442,-0.021078,2.303567,11.188204,1.359159,...,2.630154,10.272152,1.717391,0.582278,05:00:00,46,79,158,370.467391,4445.608696


In [12]:
res['Sharpe ratio'].median()

2.0089953932344664

In [13]:
res['Position EV'].median()

3.1221590909090935

In [14]:
res[res['Position EV'] < 2]['Position EV'].count()

27